# Beta Bank Client CHurn Analysis

## 1. Defining the Question

### a) Data Analysis Question

Can we predict the delivery time of Sendy orders?

### b) Metric for Success

The model should predict an accurate delivery time, from picking up a package upto when it arrives at the final
destination.

### c) Understanding the context 

Sendy has hired you to help predict the estimated time of delivery of orders, from the point of driver pickup to the point of arrival at the final destination. 

Build a model that predicts an accurate delivery time, from picking up a package to arriving at the final destination. An accurate arrival time prediction will help all business to improve their logistics and communicate the accurate time their time to their customers.

You will be required to perform various feature engineering techniques while preparing your data for further analysis.

### d) Experimental Design

1. Defining the Research Question
2. Data Importation
3. Data Exploration
4. Data Cleaning
5. Data Analysis (Univariate and Bivariate)
6. Data Preparation
7. Data Modeling
8. Model Evaluation
9. Challenging your Solution
10. Recommendations / Conclusion

### e) Data Relevance

The given data set is relevant in answering the research question.

## 2. Reading the Data

In [93]:
# Importing our libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
pd.set_option('display.max_columns', None)


In [94]:
# Additional packages

# importing six and sys
import six
import sys
sys.modules['sklearn.externals.six'] = six

# installing mlrose
!pip install mlrose
import mlrose

# importing joblib
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [95]:
# Load the data
# --- 
data_url = "https://raw.githubusercontent.com/wambasisamuel/DE_Week04_Friday/main/sendy_logistics.csv"
df = pd.read_csv(data_url) 

In [96]:
# Description of the data
# --- 
description_url = "https://raw.githubusercontent.com/wambasisamuel/DE_Week04_Friday/main/VariableDefinitions.csv"
data_desc = pd.read_csv(description_url)
data_desc

,Order No,Unique number identifying the order
0,User Id,Unique number identifying the customer on a pl...
1,Vehicle Type,"For this competition limited to bikes, however..."
2,Platform Type,"Platform used to place the order, there are 4 ..."
3,Personal or Business,Customer type
4,Placement - Day of Month,Placement - Day of Month i.e 1-31
5,Placement - Weekday (Mo = 1),Placement - Weekday (Monday = 1)
6,Placement - Time,Placement - Time - Time of day the order was p...
7,Confirmation - Day of Month,Confirmation - Day of Month i.e 1-31
8,Confirmation - Weekday (Mo = 1),Confirmation - Weekday (Monday = 1)
9,Confirmation - Time,Confirmation - Time - Time of day the order wa...


In [97]:
# Checking the first 5 rows of data
df.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [98]:
# Checking the last 5 rows of data
# ---
df.tail(5)

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
21196,Order_No_8834,User_Id_2001,Bike,3,Personal,20,3,3:54:38 PM,20,3,3:55:09 PM,20,3,3:58:49 PM,20,3,4:20:08 PM,20,3,4:20:17 PM,3,28.6,NaN,-1.258414,36.804800,-1.275285,36.802702,Rider_Id_953,9
21197,Order_No_22892,User_Id_1796,Bike,3,Business,13,6,10:13:34 AM,13,6,10:13:41 AM,13,6,10:20:04 AM,13,6,10:33:27 AM,13,6,10:46:17 AM,7,26.0,NaN,-1.307143,36.825009,-1.331619,36.847976,Rider_Id_155,770
21198,Order_No_2831,User_Id_2956,Bike,3,Business,7,4,5:06:16 PM,7,4,5:07:09 PM,7,4,5:30:17 PM,7,4,5:50:52 PM,7,4,6:40:05 PM,20,29.2,NaN,-1.286018,36.897534,-1.258414,36.804800,Rider_Id_697,2953
21199,Order_No_6174,User_Id_2524,Bike,1,Personal,4,3,9:31:39 AM,4,3,9:31:53 AM,4,3,9:38:59 AM,4,3,9:45:15 AM,4,3,10:08:15 AM,13,15.0,NaN,-1.250030,36.874167,-1.279210,36.794872,Rider_Id_347,1380
21200,Order_No_9836,User_Id_718,Bike,3,Business,26,2,2:19:47 PM,26,2,2:20:01 PM,26,2,2:24:29 PM,26,2,2:41:55 PM,26,2,3:17:23 PM,12,30.9,NaN,-1.255189,36.782203,-1.320157,36.830887,Rider_Id_177,2128


In [99]:
# Checking number of rows and columns
df.shape

(21201, 29)

In [100]:
# Checking datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Order No                                   21201 non-null  object 
 1   User Id                                    21201 non-null  object 
 2   Vehicle Type                               21201 non-null  object 
 3   Platform Type                              21201 non-null  int64  
 4   Personal or Business                       21201 non-null  object 
 5   Placement - Day of Month                   21201 non-null  int64  
 6   Placement - Weekday (Mo = 1)               21201 non-null  int64  
 7   Placement - Time                           21201 non-null  object 
 8   Confirmation - Day of Month                21201 non-null  int64  
 9   Confirmation - Weekday (Mo = 1)            21201 non-null  int64  
 10  Confirmation - Time   

Observations:

*   The are 21201 observations in the dataset.
*   The dataset has 29 features.
*   There are 10 categorical features
*   There are 19 numerical features



## 3. External Data Source Validation

The provided dataset has enough features to help in developing a machine learning model that can predict delivery time.

## 4. Data Preparation

### Data Standardisation

In [101]:
# Standardise column names
# ---
df.columns = df.columns.str.strip().str.lower().str.replace(" ","_")
df.columns

Index(['order_no', 'user_id', 'vehicle_type', 'platform_type',
       'personal_or_business', 'placement_-_day_of_month',
       'placement_-_weekday_(mo_=_1)', 'placement_-_time',
       'confirmation_-_day_of_month', 'confirmation_-_weekday_(mo_=_1)',
       'confirmation_-_time', 'arrival_at_pickup_-_day_of_month',
       'arrival_at_pickup_-_weekday_(mo_=_1)', 'arrival_at_pickup_-_time',
       'pickup_-_day_of_month', 'pickup_-_weekday_(mo_=_1)', 'pickup_-_time',
       'arrival_at_destination_-_day_of_month',
       'arrival_at_destination_-_weekday_(mo_=_1)',
       'arrival_at_destination_-_time', 'distance_(km)', 'temperature',
       'precipitation_in_millimeters', 'pickup_lat', 'pickup_long',
       'destination_lat', 'destination_long', 'rider_id',
       'time_from_pickup_to_arrival'],
      dtype='object')

### Data Cleaning

#### Irrelevant Data

The columns *order_no, user_id* and *rider_id* serve as identifier variables and serve no major purpose in modelling, hence I will remove them.

In [102]:
df.drop(columns=['order_no', 'user_id', 'rider_id'],inplace=True)

In [103]:
vehicle_type_count = df['vehicle_type'].value_counts()
vehicle_type_count

Bike    21201
Name: vehicle_type, dtype: int64

There is only one vehicle type, hence it won't have an impact on modelling.

In [104]:
df.drop(columns=['vehicle_type'],inplace=True)

In my opinion, the time related variables that are relevant in determining delivery time, are those that measure time from the moment an Order is picked up. Hence, I will keep a single 'Day of Month', 'Weekday' and 'Time' column for 'Pickup'.

In [105]:
time_cols_drop = ['confirmation_-_day_of_month', 'confirmation_-_weekday_(mo_=_1)', \
           'confirmation_-_time', 'arrival_at_pickup_-_day_of_month', 'arrival_at_pickup_-_weekday_(mo_=_1)', \
           'arrival_at_pickup_-_time','pickup_-_day_of_month', 'pickup_-_weekday_(mo_=_1)']

df.drop(time_cols_drop, inplace=True, axis=1)

#### Duplicate data

In [106]:
# Find the total duplicate records
df.duplicated().sum()

0

#### Missing Data

In [107]:
# Checking missing entries of all the variables
# ---
# 
df.isnull().sum()

platform_type                                    0
personal_or_business                             0
placement_-_day_of_month                         0
placement_-_weekday_(mo_=_1)                     0
placement_-_time                                 0
pickup_-_time                                    0
arrival_at_destination_-_day_of_month            0
arrival_at_destination_-_weekday_(mo_=_1)        0
arrival_at_destination_-_time                    0
distance_(km)                                    0
temperature                                   4366
precipitation_in_millimeters                 20649
pickup_lat                                       0
pickup_long                                      0
destination_lat                                  0
destination_long                                 0
time_from_pickup_to_arrival                      0
dtype: int64

Since Nairobi is arid to semi-arid, precipitation in this case will refer to rainfall. Rainfall affects traffic and road confitions, hence I won't remove this variable. I will replace the null values with 0 mm of rainfall, assuming that the missing values imply no rainfall.

In [108]:
df['precipitation_in_millimeters'].fillna(0,inplace=True)

I will replace the missing values in the temperature values with the mean

In [109]:
# I will replace null values in the `temperature` column with the mean
df['temperature'].fillna(value=df['temperature'].mean(), inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21201 entries, 0 to 21200
Data columns (total 17 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   platform_type                              21201 non-null  int64  
 1   personal_or_business                       21201 non-null  object 
 2   placement_-_day_of_month                   21201 non-null  int64  
 3   placement_-_weekday_(mo_=_1)               21201 non-null  int64  
 4   placement_-_time                           21201 non-null  object 
 5   pickup_-_time                              21201 non-null  object 
 6   arrival_at_destination_-_day_of_month      21201 non-null  int64  
 7   arrival_at_destination_-_weekday_(mo_=_1)  21201 non-null  int64  
 8   arrival_at_destination_-_time              21201 non-null  object 
 9   distance_(km)                              21201 non-null  int64  
 10  temperature           

I will encode the personal_or_business variable

In [110]:
df['personal_or_business'] = df['personal_or_business'].astype('category').cat.codes
df['personal_or_business'].value_counts()

0    17384
1     3817
Name: personal_or_business, dtype: int64

I will convert the date/time columns to the format HMS e.g 3:54:38 PM becomes 155438

In [111]:
from datetime import datetime

def convert_time(time):
  formatted_time = datetime.strptime(time, "%I:%M:%S %p")
  str_from_time = datetime.strftime(formatted_time, "%H%M%S")
  return str_from_time

time_columns = ['placement_-_time', 'pickup_-_time', 'arrival_at_destination_-_time']

for col in time_columns:
  df[col] = df[col].apply(convert_time)

df.sample(5)

,platform_type,personal_or_business,placement_-_day_of_month,placement_-_weekday_(mo_=_1),placement_-_time,pickup_-_time,arrival_at_destination_-_day_of_month,arrival_at_destination_-_weekday_(mo_=_1),arrival_at_destination_-_time,distance_(km),temperature,precipitation_in_millimeters,pickup_lat,pickup_long,destination_lat,destination_long,time_from_pickup_to_arrival
16381,1,1,15,2,090156,092519,15,2,094042,5,18.100000,2.0,-1.304696,36.801646,-1.317867,36.812166,923
12907,3,0,26,5,162825,163646,26,5,164855,4,23.258889,0.0,-1.276091,36.808103,-1.257147,36.795063,729
3035,3,0,8,5,133603,141616,8,5,153638,7,27.000000,0.0,-1.286177,36.820909,-1.255189,36.782203,4822
91,3,0,28,4,151047,152507,28,4,153735,5,30.100000,0.0,-1.260975,36.788663,-1.274396,36.821591,748
587,3,0,1,5,122003,123554,1,5,124951,8,26.700000,0.0,-1.255189,36.782203,-1.285417,36.795115,837


## 5. Data Modelling

#### Base model - Ensemble Regressor

In [112]:
features = df.drop(['time_from_pickup_to_arrival'], axis=1)
target = df['time_from_pickup_to_arrival']

# Splitting the data into training, testing and validation sets. Of the base dataset: 20% - testing, 80% - training.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)

# Model 
base_regressor = RandomForestRegressor(n_estimators = 10, random_state = 12345)

# Train the model
base_regressor.fit(features_train, target_train)

# Predict using the model
predictions = base_regressor.predict(features_test)

# RMSE
rmse = mean_squared_error(target_test, predictions) ** 0.5

print('RMSE: ', rmse)

RMSE:  525.3969582871707


## 5. Model Optimization

#### Feature Scaling

Standardization

In [114]:
# Scaler
scaler = StandardScaler()

# Scaled model
features_train_scaled = scaler.fit_transform(features_train) 
features_test_scaled = scaler.fit_transform(features_test)

# fit the model
regressor_scaled = RandomForestRegressor(n_estimators = 10, random_state = 12345)
regressor_scaled.fit(features_train_scaled, target_train)

# predict
predictions_scaled = regressor_scaled.predict(features_test_scaled)

rmse = np.sqrt(mean_squared_error(target_test, predictions_scaled))

print('RMSE: ', rmse)



RMSE:  526.5920410670499


Normalization

In [113]:
# Normalized model
normalized = MinMaxScaler().fit(features_train) 
features_train_normalized = normalized.transform(features_train) 
features_test_normalized = normalized.transform(features_test)

# fit the model
regressor_normalized = RandomForestRegressor(n_estimators = 10, random_state = 12345)
regressor_normalized.fit(features_train_normalized, target_train)

# predict
predictions_normalized = regressor_normalized.predict(features_test_normalized)

# RMSE
rmse = np.sqrt(mean_squared_error(target_test, predictions_normalized))

print('RMSE:', rmse)

RMSE: 525.5213235397118


Normalization achieves better result than standardization

#### Feature Selection

**Step Forward Feature Selection**

In [ ]:
features = df.drop(['time_from_pickup_to_arrival'], axis=1)
target = df['time_from_pickup_to_arrival']

# Splitting the data into training, testing and validation sets. Of the base dataset: 20% - testing, 80% - training.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)

# Normalized model
normalized = MinMaxScaler().fit(features_train) 
features_train_normalized = normalized.transform(features_train) 
features_test_normalized = normalized.transform(features_test)

regressor_sf = RandomForestRegressor(n_estimators = 10, random_state = 12345)

# I'll pass the normalised model defined above to the SequentialFeatureSelector function. 
# k_features: number of features to select. 
# forward: if set to True, performs step forward feature selection. 
# verbose: used for logging the progress of the feature selector
# scoring: defines the performance evaluation criteria 
# cv: refers to cross-validation folds.

feature_selector_sf = SequentialFeatureSelector(regressor_sf,
           k_features=15,
           forward=True,
           verbose=2,
           scoring='r2',
           cv=4)
 
# Step forward feature selection
feature_selector_sf = feature_selector_sf.fit(features_train_normalized, target_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    8.2s finished

[2022-11-27 20:28:05] Features: 1/15 -- score: 0.3391752466522554[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:    7.3s finished

[2022-11-27 20:28:12] Features: 2/15 -- score: 0.33770154263715374[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  14 out of  14 | elapsed:    7.8s finished

[2022-11-27 20:28:20] Features: 3/15 -- score: 0.33470381059571186[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done

In [118]:
selected_features_sb = list(feature_selector_sf.k_feature_idx_)
#selected_features

# Modelling with features from step forward selection
step_forward_regressor = RandomForestRegressor(n_estimators = 10, random_state = 12345)
step_forward_regressor.fit(features_train_normalized[:, selected_features_sb], target_train)

# Making Predictions and determining the accuracies
step_forward_predictions = step_forward_regressor.predict(features_test_normalized[:, selected_features_sb])

# RMSE
rmse = np.sqrt(mean_squared_error(target_test, step_forward_predictions))

print('RMSE with step forward features: ', rmse)

RMSE with step forward features:  490.43403365549784


**Step Backward Feature Selection**

In [119]:
# This time I set the `forward` parameter to False

feature_selector_sb = SequentialFeatureSelector(regressor_normalized,
           k_features=15,
           forward=False,
           verbose=2,
           scoring='r2',
           cv=4)
 
# Step forward feature selection
feature_selector_sb = feature_selector_sb.fit(features_train_normalized, target_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:  1.3min finished

[2022-11-27 20:04:35] Features: 15/15 -- score: 0.6827128528080655

In [121]:
selected_features_sb = list(feature_selector_sb.k_feature_idx_)
#selected_features

# Modelling with features from step backward selection
step_backward_regressor = RandomForestRegressor(n_estimators = 10, random_state = 12345)
step_backward_regressor.fit(features_train_normalized[:, selected_features_sb], target_train)

# Making Predictions and determining the accuracies
step_backward_predictions = step_backward_regressor.predict(features_test_normalized[:, selected_features_sb])

# RMSE
rmse = np.sqrt(mean_squared_error(target_test, step_backward_predictions))

print('RMSE with step backward features: ', rmse)

RMSE with step backward features:  490.43403365549784


RMSE improves from 522 to 490

**Recursive Feature Elimination**

In [89]:
# Checking missing entries of all the variables
# ---
# 
df.isnull().sum()

platform_type                                0
personal_or_business                         0
placement_-_day_of_month                     0
placement_-_weekday_(mo_=_1)                 0
placement_-_time                             0
pickup_-_time                                0
arrival_at_destination_-_day_of_month        0
arrival_at_destination_-_weekday_(mo_=_1)    0
arrival_at_destination_-_time                0
distance_(km)                                0
temperature                                  0
precipitation_in_millimeters                 0
pickup_lat                                   0
pickup_long                                  0
destination_lat                              0
destination_long                             0
rider_id                                     0
time_from_pickup_to_arrival                  0
dtype: int64

#### Data Preprocessing

Transforming categorical features into numerical features.

In [ ]:
df_ohe = pd.get_dummies(df, drop_first=True)

Splitting the dataset

In [ ]:
# Features and Targets
features = df_ohe.drop(columns=['exited'])
target =  df_ohe['exited']

# Splitting the data into training, testing and validation sets. Of the base dataset: 20% - testing, 80% - training.
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.20, random_state=12345)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.2, random_state=12345 )

Standardize numerical features

In [ ]:
# Standardize numerical features
numeric_features = ['creditscore', 'age', 'balance', 'estimatedsalary', 'tenure', 'numofproducts']
scaler = StandardScaler()
scaler.fit(features_train[numeric_features])
features_train.loc[:,numeric_features] = scaler.transform(features_train[numeric_features])
features_valid.loc[:,numeric_features] = scaler.transform(features_valid[numeric_features])
features_test.loc[:,numeric_features] = scaler.transform(features_test[numeric_features])


#### Class Balance

In [ ]:
class_freq = df_ohe['exited'].value_counts(normalize= True)
print(class_freq)
class_freq.plot(kind='bar')
plt.show()

There is imbalance of classes.

#### Sanity check


I will create a model which always predicts value 0 and see it's accuracy and F1 score

In [ ]:
target_pred_constant = pd.Series(0, index=range(target_valid.shape[0]))

print(accuracy_score(target_valid, target_pred_constant))
print('F1:', f1_score(target_valid, target_pred_constant))

#### Modelling with Imbalance

In [ ]:
log_reg_imb = LogisticRegression(solver='liblinear', random_state=12345)
log_reg_imb.fit(features_train,target_train)
predicted_valid = log_reg_imb.predict(features_valid)

print('Accuracy: ', log_reg_imb.score(features_valid, target_valid))
print('F1 score: ', f1_score(target_valid, predicted_valid))
print('F1 score: ',  roc_auc_score(target_valid, log_reg_imb.predict_proba(features_valid)[:,1]))



Without accounting for class imbalance, logistic regression achieves an accuracy of 0.81, f1 score of 0.31, and AUC of 0.76.

Random Forest and Decision Tree models are affected with class imbalance as well, hence I won't check for these models.


#### Accounting for Imbalance

In [ ]:
log_reg_bal = LogisticRegression(random_state=12345, class_weight='balanced', solver='liblinear')
log_reg_bal.fit(features_train,target_train)
predicted_valid = log_reg_bal.predict(features_valid)

print('Accuracy: ', log_reg_bal.score(features_valid, target_valid))
print('F1 score: ', f1_score(target_valid, predicted_valid))
print('AUC: ',  roc_auc_score(target_valid, log_reg_bal.predict_proba(features_valid)[:,1]))

After accounting for imbalance, we achieve accuracy of 0.70, f1 score of 0.47, and AUC of 0.77. The f1 score has improved at te expense of accuracy.

#### Upsampling

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

log_reg_upsampled =LogisticRegression(random_state=12345,solver='liblinear')
log_reg_upsampled.fit(features_upsampled, target_upsampled)
log_reg_upsampled_predicted_valid = log_reg_upsampled.predict(features_valid)

print('Accuracy', log_reg_upsampled.score(features_valid, target_valid))
print('f1 score:' ,f1_score(target_valid, log_reg_upsampled_predicted_valid))
print('AUC:',roc_auc_score(target_valid, log_reg_upsampled.predict_proba(features_valid)[:,1]))

 Upsampling gives an F1 score of 0.3927

#### Downsampling

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.1)

log_reg_downsampled =LogisticRegression(random_state=12345,solver='liblinear')
log_reg_downsampled.fit(features_downsampled, target_downsampled)
log_reg_downsampled_predicted_valid = log_reg_downsampled.predict(features_valid)

print("F1:", f1_score(target_valid, log_reg_downsampled_predicted_valid))
print('Accuracy:', log_reg_downsampled.score(features_valid, target_valid))
print("AUC-ROC:", roc_auc_score(target_valid, log_reg_downsampled.predict_proba(features_valid)[:,1]))

Downsampling gives an F1 score of 0.3920

#### Decision Tree

In [ ]:
for depth in range(1,15):
  model =  DecisionTreeClassifier(random_state = 12345, max_depth = depth)
  model.fit(features_train, target_train)
  predicted_valid = model.predict(features_valid)
  print('\n========Depth = ',depth,' ========')
  print('Accuracy:' , accuracy_score(target_valid, predicted_valid))
  print('F1:', f1_score(target_valid, predicted_valid))
  print("AUC-ROC:", roc_auc_score(target_valid, model.predict_proba(features_valid)[:,1]))

A decision tree model with optimized hyper parameters (depth=6) gives an F1 score of 0.5177

#### Random Forest

In [ ]:
params = {'max_depth':range(1,10), 'n_estimators':range(1,50)}
rand_frst_model = RandomForestClassifier(random_state=12345)
rand_frst_optimized = GridSearchCV(rand_frst_model, params)

# Train using the optimized model
rand_frst_optimized.fit(features_train, target_train)

print(rand_frst_optimized.best_estimator_)

# Predict using the optimized model
predicted_valid = rand_frst_optimized.predict(features_valid)

print('Accuracy', rand_frst_optimized.score(features_valid, target_valid))
print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, rand_frst_optimized.predict_proba(features_valid)[:,1]))

The random forest model gives an F1 score of 0.5572; however, this takes a very long time to run. We are yet to achieve an F1 score of at least 0.59. The max_depth of 8 gives the best result. I will keep this parameter constant and increase the range of n_estimators and investigate the effect on perfomance. I will add the parameter 'class weight = balanced'

In [ ]:
params = {'n_estimators':range(1,201)}
rand_frst_model = RandomForestClassifier(random_state=12345, max_depth = 8,class_weight='balanced')
rand_frst_optimized = GridSearchCV(rand_frst_model, params)

# Train using the optimized model
rand_frst_optimized.fit(features_train, target_train)

print(rand_frst_optimized.best_estimator_)

# Predict using the optimized model
predicted_valid = rand_frst_optimized.predict(features_valid)

print('Accuracy', rand_frst_optimized.score(features_valid, target_valid))
print("F1:", f1_score(target_valid, predicted_valid))
print("AUC-ROC:", roc_auc_score(target_valid, rand_frst_optimized.predict_proba(features_valid)[:,1]))

190 n_estimators are needed to produce the final optimized model which gives an F1 score of 0.606 

I will use this model for testing.

In [ ]:
predicted_test = rand_frst_optimized.predict(features_test)
print("F1:", f1_score(target_test, predicted_test))
print("AUC-ROC:", roc_auc_score(target_test, rand_frst_optimized.predict_proba(features_test)[:,1]))
print('Accuracy:', rand_frst_optimized.score(features_valid, target_valid))

Applying the optimized model on the test dataset gives an F1 score of 0.6474 and the AUC-ROC score of 0.8693. These meet the requirements for this project.

## 6. Conclusion

#### AUC-ROC Curve

In [ ]:
valid_probabilities = rand_frst_optimized.predict_proba(features_valid)
valid_ones_probabilities = valid_probabilities[:, 1]

fpr, tpr, thresholds = roc_curve(target_valid, valid_ones_probabilities)

plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.show()

#### Findings

A Random Forest Classifier is the best model. Bootstrap is set to true and the class_weight parameter is set to '*balanced*

The hyperparameters are set as follows:

* random_state = 12345
* max_depth = 8
* n_estimators = 90

Applying the final optimized model on the test dataset, gives an F1 score of 0.6474 and the AUC-ROC score is 0.8693. Both these metrics signify good quality and meet the expectations of the project.



